In [1]:
import numpy as np
import pandas as pd
import random

from scipy import stats
from sklearn.ensemble import HistGradientBoostingRegressor
import statistics as st
import matplotlib as mpl
mpl.use("PS")
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.font_manager import FontProperties

mpl.rcParams['figure.dpi'] = 900
mpl.rcParams['ytick.labelsize'] = 8

In [18]:
### Set hyperparameters
RESET = True  # Reset the internal state (H) at transition from the last TRAIN to the First TEST
FIRST_ACTION_INTAKE = 1
# 0 : The initial intakes of each episode reflects the Qs and radom number.
# 1 : The initial action of the Train 1 is definetely intake, but the others are based on Qs and randon number.
# 2 : The all initial actions of all trains and tests are definetely intake.

'''
ALPHA, BETA, GAMMA = 0.009, 3, 0.7  # learning rate of Q, inverse temperature, discounting rate
MOUTHS_L = [0.7, 2.5, 4.5]  # The amount for an intake [water (every selectable actions), 300mM salt, 750mM salt]
H_ast, H_full, H_dep = 100, 100, 80  # The setpoint, the full state, the depleted state
TAUS_L = [90000, 70000]  # attenuating rates of Hs [water, sodium]
m, n = 1, 2
'''

ALPHA, BETA, GAMMA = 0.001, 0.05, 0.7  # learning rate of Q, inverse temperature, discounting rate
MOUTHS_L = [22, 35, 100]  # The amount for an intake [water (every selectable actions), 300mM salt, 750mM salt]
H_ast, H_full, H_dep = 1000, 1000, 700  # The setpoint, the full state, the depleted state
TAUS_L = [7000, 5000]  # attenuating rates of Hs [water, sodium]
m, n = 1, 2


TIMESTEPS_N, TRIALS_TRAIN_N, TRIALS_TEST_N = 40, 5, 2
# Number of timesteps in 1 trial, number of trials of train, number of trials of test

ANIMALS_N = 100  # Number of animals of the same condition

frag, fs_small, fs, rd = 10, 8, 9, 0.1  # Timesteps of 1 fragment to get the means of licking number# Font-size, density of error region color

# serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
ser_num = 0

inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

Q = np.zeros((2, (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + TRIALS_TEST_N)))  # Action values
H = np.zeros((2, (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + TRIALS_TEST_N)))  # Internal states: [water, sodium]

# test=0: water train; init_W=H_dep, init_Na=H_ast, lick='water'# test=1: salt train; init_W=H_dep, init_Na=H_ast, lick='300 mM'
# test=2.1: water test; init_W=H_ast, init_Na=0, lick='water'# test=2.2: water test; init_W=H_ast, init_Na=0, lick='300mM'# test=2.3: water test; init_W=H_ast, init_Na=0, lick='750mM'
# test=3.1: salt test; init_W=0, init_Na=H_ast, lick='water'# test=3.2: salt test; init_W=0, init_Na=H_ast, lick='300mM'# test=3.3: salt test; init_W=0, init_Na=H_ast, lick='750mM'

In [19]:
class HRLforNaWater:
    def __init__(self, test=0, reset=True):  # test is in (0, 1, 2.1, 2.2, 2.3, 3.1, 3.2, 3.3)
        self.test, self.reset = test, reset
        self.N = TIMESTEPS_N
        self.a = np.zeros(self.N)  # from [a0, a1]
        self.Intake = np.zeros(self.N)  # from [0; No intake, 1; Water intake, 2; 300mM NaCl intake, 3; 750mM NaCl intake]
        self.D, self.D_est = np.zeros(self.N), np.zeros(self.N)  # D(Ht), D(Ht+1)
        self.H_ast, self.H_full, self.H_dep = H_ast, H_full, H_dep  # The setpoint, the full state, the depleted state
        self.first_action_intake = FIRST_ACTION_INTAKE

        # Initialize H
        if self.reset == True:
            if ser_num == 0 or ser_num == (TIMESTEPS_N + 1) * TRIALS_TRAIN_N:
            # The initial timestep of the first trains or tests.
                if self.test in (0, 2.1, 2.2, 2.3):  # water train or water tests
                    H[0][ser_num], H[1][ser_num] = self.H_dep, self.H_full  # init_W, init_Na
                elif self.test in (1, 3.1, 3.2, 3.3):  # salt train or water tests
                    H[0][ser_num], H[1][ser_num] = self.H_full, self.H_dep  # init_W, init_Na
            else:
                if ser_num % (TIMESTEPS_N + 1) == 0:
                # The initial timesteps of each trains and tests except for of the first trains and tests.
                    if self.test in (0, 1):  # trains
                        H[0][ser_num], H[1][ser_num] = inheritH_inTRAIN[0][-1], inheritH_inTRAIN[1][-1]  # water, sodium
                    else:  # all tests
                        H[0][ser_num], H[1][ser_num] = inheritH_inTEST[0][-1], inheritH_inTEST[1][-1]  # water, sodium

        elif self.reset == False:
            if ser_num == 0:
            # The initial timestep of the first trains or tests.
                if self.test in (0, 2.1, 2.2, 2.3):  # water train or water tests
                    H[0][ser_num], H[1][ser_num] = self.H_dep, self.H_full  # init_W, init_Na
                elif self.test in (1, 3.1, 3.2, 3.3):  # salt train or water tests
                    H[0][ser_num], H[1][ser_num] = self.H_full, self.H_dep  # init_W, init_Na
            elif ser_num == (TIMESTEPS_N + 1) * TRIALS_TRAIN_N:
                H[0][ser_num], H[1][ser_num] = inheritH_TRAINtoTest[0][-1], inheritH_TRAINtoTest[1][-1]  # water, sodium
            else:
                if ser_num % (TIMESTEPS_N + 1) == 0:
                # The initial timesteps of each trains and tests except for the first trains and tests.
                    if self.test in (0, 1):  # trains
                        H[0][ser_num], H[1][ser_num] = inheritH_inTRAIN[0][-1], inheritH_inTRAIN[1][-1]  # water, sodium
                    else:  # all tests
                        H[0][ser_num], H[1][ser_num] = inheritH_inTEST[0][-1], inheritH_inTEST[1][-1]  # water, sodium

        # Initialize Q
        if ser_num % (TIMESTEPS_N + 1) == 0:
            if ser_num < (TIMESTEPS_N + 1) * TRIALS_TRAIN_N:  # TRAIN to TRAIN
                Q[0][ser_num], Q[1][ser_num] = inheritQ_inTRAIN[0][-1], inheritQ_inTRAIN[1][-1]  # Do nothing, Lick
            elif ser_num == (TIMESTEPS_N + 1) * TRIALS_TRAIN_N:  # TRAIN toTEST
                Q[0][ser_num], Q[1][ser_num] = inheritQ_TRAINtoTEST[0][-1], inheritQ_TRAINtoTEST[1][-1]  # Do nothing, Lick
            else:  # TEST to TEST
                Q[0][ser_num], Q[1][ser_num] = inheritQ_inTEST[0][-1], inheritQ_inTEST[1][-1]  # Do nothing, Lick

        self.P, self.K = np.zeros((2, self.N)), np.zeros((2, self.N))  # P of all actions, an intake amount
        self.R, self.TDerror = np.zeros(self.N), np.zeros(self.N)# Reward of all timesteps, TD errors to visualize
                
        self.m, self.n = m, n  # Constants of the homeostatic space
        self.alpha, self.beta, self.gamma = ALPHA, BETA, GAMMA
        self.tau, self.mouth = TAUS_L, MOUTHS_L  # attenuation rate of the internal states, volume of an intake


    def softmax(self, t):  # from Qs to Ps
        self.t = t
        sigma = np.sum(np.exp(Q[:, ser_num] * self.beta))
        vals = []

        for i in range(2):
            softmax = np.exp(Q[i, ser_num] * self.beta) / sigma
            vals.append(softmax)
        
        # Actions (0...Do nothing; 1...intake)
        self.P[0][self.t], self.P[1][self.t] = (softmax, 1 - softmax) if i == 0 else (1 - softmax, softmax)

        dice = random.random()

        self.a[self.t] = 0 if dice <= vals[0] else 1
        if (self.first_action_intake == 2 and self.t == 0) or (self.first_action_intake == 1 and ser_num == 0):
            self.a[self.t] = 1


    def get_K(self, t):
        self.t = t
        if self.t == 0:  # The initial lick
            if self.test == 0:  # water train
                if self.a[self.t] == 0:  # Do nothing
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                elif self.a[self.t] == 1:  # Water intake
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = self.mouth[0], 0, 1
            elif self.test == 1:  # salt train
                if self.a[self.t] == 0:  # Do nothing
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                elif self.a[self.t] == 1:  # 300mM intake
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = self.mouth[0], self.mouth[1], 2
            elif self.test == 2.1 or self.test == 3.1:  # both test, water lick
                if self.a[self.t] == 0:  # Do nothing
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                elif self.a[self.t] == 1:  # Water
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = self.mouth[0], 0, 1
            elif self.test == 2.2 or self.test == 3.2:  # both test, 300mM lick
                if self.a[self.t] == 0:  # Do nothing
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                elif self.a[self.t] == 1:  # 300mM
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = self.mouth[0], self.mouth[1], 2
            elif self.test == 2.3 or self.test == 3.3:  # both test, 750mM lick
                if self.a[self.t] == 0:  # Do nothing
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                elif self.a[self.t] == 1:  # 750mM
                    self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = self.mouth[0], self.mouth[2], 3

        elif self.t != 0:  # Other than the initial lick of episode
            if self.a[self.t] == 0:  # Do nothing
                self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
            elif self.a[self.t] == 1:  # Lick
                self.K[0][self.t], self.K[1][self.t], self.Intake[self.t] = 0, 0, 0
                

    def get_D(self, t):
        self.t = t
        self.D[self.t] =  (abs(self.H_ast - H[0][ser_num]) ** self.n
                           + abs(self.H_ast - H[1][ser_num]) ** self.n) ** (1 / self.m)
        self.D_est[self.t] = (abs(self.H_ast - (1 - 1 / self.tau[0]) * H[0][ser_num] - self.K[0][self.t]) ** self.n
                                 + abs(self.H_ast - (1 - 1 / self.tau[1]) * H[1][ser_num] - self.K[1][self.t]) ** self.n) ** (1 / self.m)
        

    def get_R(self, t):
        self.t = t
        self.R[self.t] = self.D[self.t] - self.D_est[self.t]
        

    def get_H(self, t):
        self.t = t
        H[0][ser_num + 1], H[1][ser_num + 1] = (1 - 1 / self.tau[0]) * H[0][ser_num] + self.K[0][self.t], (1 - 1 / self.tau[1]) * H[1][ser_num] + self.K[1][self.t]
        
        
    def get_Q(self, t):  # Q-learning
        self.t = t
        if self.a[self.t] == 0:
            self.TDerror[self.t] = self.R[self.t] + self.gamma * max(Q[0][ser_num], Q[1][ser_num]) - Q[0][ser_num]
            Q[0][ser_num + 1], Q[1][ser_num + 1] = Q[0][ser_num] + self.alpha * self.TDerror[self.t], Q[1][ser_num]
        elif self.a[self.t] == 1:
            self.TDerror[self.t] = self.R[self.t] + self.gamma * max(Q[0][ser_num], Q[1][ser_num]) - Q[1][ser_num]
            Q[0][ser_num + 1], Q[1][ser_num + 1] = Q[0][ser_num], Q[1][ser_num] + self.alpha * self.TDerror[self.t]



In [20]:
def visualize(cond, A_H0, A_H1, A_Q0, A_Q1, A_P0, A_P1, A_Intake, A_a, A_R, A_TDerror, A_D):
    
    df = pd.DataFrame(data=list(range(TIMESTEPS_N + 1)) * (TRIALS_TRAIN_N + TRIALS_TEST_N), columns=['trial'])

    df['H0'], df['H1'] = A_H0, A_H1
    df['Q0'], df['Q1'] = A_Q0, A_Q1
    df['P0'], df['P1'] = A_P0, A_P1

    df['Intake'], df['a'] = A_Intake, A_a
    df['R'], df['TDerror'], df['D'] = A_R, A_TDerror, A_D

    visu_variables_n = 10
    fig = plt.figure(figsize=(8, 8))
    height_ratios = [3]*(visu_variables_n-1)
    height_ratios.append(1)
    spec = gridspec.GridSpec(ncols=1, nrows=visu_variables_n, height_ratios=height_ratios)

    ax_l = []
    for i in range(visu_variables_n):
        ax_l.append(fig.add_subplot(spec[i]))

    Df = TRIALS_TEST_N - 1  # degree of freedom for Figures
    mean = []  # mean at t

    for j, h in zip(['H0','H1','Q0','Q1','P1','R','TDerror','D'], list(range(visu_variables_n))):
        for i in range((TIMESTEPS_N + 1) * 2):
            mean.append(sum(df.loc[df['trial']==i][j]) / TRIALS_TEST_N)
            
    length_l, length_s = (TRIALS_TEST_N + 1) * 2, TRIALS_TEST_N * 2


    ax_i = 0
    ax_l[ax_i].hlines(H_ast, 0, len(df['H0']), linewidth=1, linestyle="dashed", color='gray')
    #ax_l[ax_i].hlines(H_dep, 0, len(df['H0']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].plot(df['H0'], label='Water state', linewidth=1, color='Blue')
    ax_l[ax_i].plot(df['H1'], label='Sodium state', linewidth=1, color='#008000')
    ax_l[ax_i].set_ylabel('Internal\nstate\n(H)', fontsize=fs)
    ax_l[ax_i].set_yticks([700, 1000])
    ax_l[ax_i].set_yticklabels([r'$H_{dep} = 700$', r'$H^{*} = 1000$'], fontsize=fs_small)
    #ax_l[ax_i].legend(fontsize=fs_small, loc='best', frameon=False)
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    ax_l[ax_i].scatter(list(range(len(df['Intake']))), df['Intake'], s=10, c='black')
    ax_l[ax_i].set_ylabel('Intake', fontsize=fs)
    ax_l[ax_i].set_yticks([0, 1, 2, 3])
    ax_l[ax_i].hlines(0, 0, len(df['Intake']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].hlines(1, 0, len(df['Intake']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].hlines(2, 0, len(df['Intake']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].hlines(3, 0, len(df['Intake']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].tick_params(labelbottom=False)
    ax_l[ax_i].set_yticklabels([r'$K = (0, 0)$', r'$Water$', r'$300 mM$', r'$750 mM$'], fontproperties=FontProperties(size=fs_small))
    ax_l[ax_i].set_ylim(ax_l[ax_i].get_ylim()[::-1])
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    ax_l[ax_i].hlines(0, 0, len(df['R']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].plot(df['R'], linewidth=1, color='black')
    ax_l[ax_i].set_ylabel('Reward\n(R)' , fontsize=fs)
    ax_l[ax_i].set_yticks([-15000, 0, 15000])
    ax_l[ax_i].set_yticklabels([-15000, 0, 15000], fontsize=fs_small)
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    #ax_l[ax_i].hlines(0, 0, len(df['Q0']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].plot(df['Q0'], label='Q_0', linewidth=1, color='black')
    ax_l[ax_i].plot(df['Q1'], label='Q_1', linewidth=1, color='#FF4500')
    ax_l[ax_i].set_ylabel('Action\nstate\nvalue\n(Q)' , fontsize=fs)
    #ax_l[ax_i].legend(fontsize=fs_small, loc='best', frameon=False)
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    #ax_l[ax_i].hlines(0, 0, len(df['P1']), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].hlines(0.5, 0, len(df['P1']), linestyle="dashed", linewidth=1, color='gray')
    #ax_l[ax_i].hlines(1, 0, len(df['P1']), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].plot(df['P1'], linewidth=1, color='#FF4500')
    ax_l[ax_i].set_ylabel('Prob.\nintake\n(P)' , fontsize=fs)
    ax_l[ax_i].set_yticks([0, 1])
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    ax_l[ax_i].scatter(list(range(len(df['a']))), df['a'], s=10, color='#404040')
    ax_l[ax_i].set_ylabel('Action\n(a)', fontsize=fs)
    ax_l[ax_i].set_yticks([0, 1])
    ax_l[ax_i].set_yticklabels([0, 1], fontsize=fs_small)
    ax_l[ax_i].tick_params(labelbottom=False)
    ax_l[ax_i].hlines(0, 0, len(df['Intake']), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].hlines(1, 0, len(df['Intake']), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    ax_l[ax_i].hlines(0, 0, len(df['TDerror']), linewidth=1, linestyle="dashed", color='gray')
    ax_l[ax_i].plot(df['TDerror'], linewidth=1, color='black')
    ax_l[ax_i].set_ylabel('TD error' , fontsize=fs)
    ax_l[ax_i].set_yticks([-20000, 0, 20000])
    ax_l[ax_i].set_yticklabels([-20000, 0, 20000], fontsize=fs_small)
    ax_l[ax_i].tick_params(labelbottom=False)


    sums_frag = []
    for i in range(len(df['a']) - TRIALS_TEST_N + 1):
        sums_frag.append(sum(df['a'][i + frag - 1 : i + 2 * frag - 1])/frag)

    ax_i += 1
    #ax_l[ax_i].hlines(0, 0, len(sums_frag), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].plot(sums_frag, linewidth=1, color='#FF4500')
    ax_l[ax_i].set_ylabel('Moving\nAve. Licks/\n' + str(frag) + ' time' , fontsize=fs_small)
    ax_l[ax_i].tick_params(labelbottom=False)


    ax_i += 1
    #ax_l[ax_i].hlines(0, 0, len(df['D']), linestyle="dashed", linewidth=1, color='gray')
    ax_l[ax_i].plot(df['D'], linewidth=1, color='black')
    ax_l[ax_i].set_ylabel('Drive\n(D)' , fontsize=fs)
    ax_l[ax_i].set_yticks([0, 100000])
    ax_l[ax_i].set_yticklabels([0, 100000], fontsize=fs_small)
    ax_l[ax_i].tick_params(labelbottom=False)

    
    ax_i += 1
    y_text = 0
    for i in range(TRIALS_TRAIN_N):
        start = (TIMESTEPS_N + 1) * i
        end = (TIMESTEPS_N + 1) * (i + 1) - 2
        ax_l[ax_i].axvspan(start, end, facecolor='black', edgecolor='none', linewidth=1)
        ax_l[ax_i].text(start+10, y_text+1/4, 'train', color='white')

    for i in range(TRIALS_TEST_N):
        start = (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + i)
        end = (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + i + 1) - 2
        ax_l[ax_i].axvspan(start, end, facecolor='none', edgecolor='black', linewidth=1)
        ax_l[ax_i].text(start+10, y_text+1/4, 'test', color='black')

    ax_l[ax_i].spines['left'].set_visible(False)
    ax_l[ax_i].spines['right'].set_visible(False)
    ax_l[ax_i].spines['bottom'].set_visible(False)
    ax_l[ax_i].spines['top'].set_visible(False)
    ax_l[ax_i].set_yticklabels([])
    ax_l[ax_i].yaxis.set_visible(False)


    fig.subplots_adjust(left=0.3)
    fig.savefig(str(cond) + ".eps", dpi=600)


In [21]:
# Initialize empty dictionaries {animal 0: 2d numpy array, animal 1: 2d numpy array , ...}
a_H0_d, a_H1_d, a_Q0_d, a_Q1_d = {}, {}, {}, {}
a_P0_d, a_P1_d, a_a_d, a_Intake_d = {}, {}, {}, {}
a_R_d, a_TDerror_d, a_D_d = {}, {}, {}

In [22]:
# main
cond = [0, 2.1]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

In [23]:
# main
cond = [0, 2.2]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

print('water-dep, 300mM salt water test', str(cond))

water-dep, 300mM salt water test [0, 2.2]


In [24]:
# main
cond = [0, 2.3]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

print(cond, 'water-dep, 750mM salt water test', str(cond))

[0, 2.3] water-dep, 750mM salt water test [0, 2.3]


In [25]:
# main
cond = [1, 3.1]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

print(cond, 'sodium-dep, water test', str(cond))

[1, 3.1] sodium-dep, water test [1, 3.1]


In [26]:
# main
cond = [1, 3.2]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

print(cond, 'sodium-dep, 300mM salt water test', str(cond))

[1, 3.2] sodium-dep, 300mM salt water test [1, 3.2]


In [27]:
# main
cond = [1, 3.3]

# Add initial values to the data that requires aggregating data from all conditions.
a_H0_d[str(cond)], a_H1_d[str(cond)], a_Q0_d[str(cond)], a_Q1_d[str(cond)] = [], [], [], []
a_P0_d[str(cond)], a_P1_d[str(cond)], a_a_d[str(cond)], a_Intake_d[str(cond)] = [], [], [], []
a_R_d[str(cond)], a_TDerror_d[str(cond)], a_D_d[str(cond)] = [], [], []

for animal_i in range(ANIMALS_N):
    # serial number of trial (timestep from TRAIN to TEST; each episode includes 1 more element (= inhetited data)(H0, Q0, ...))
    ser_num = 0

    inheritQ_inTRAIN, inheritQ_TRAINtoTEST, inheritQ_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]
    inheritH_inTRAIN, inheritH_TRAINtoTest, inheritH_inTEST = [[0], [0]], [[0], [0]], [[0], [0]]

    num_l, trials_n_l = list(range((TIMESTEPS_N + 1) * 2)), list(range(TRIALS_TRAIN_N + TRIALS_TEST_N))

    a_H0, a_H1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_Q0, a_Q1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_P0, a_P1 = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    a_a, a_Intake = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_R, a_TDerror = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1)), np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))
    a_D = np.zeros((TRIALS_TRAIN_N + TRIALS_TEST_N) * (TIMESTEPS_N + 1))

    class_l = []  # A list storing classes
    for timestep_j in range(TRIALS_TRAIN_N + TRIALS_TEST_N):
        class_l.append(None)


    # TRAIN #################################
    ser_num = 0

    for trial_i in range(TRIALS_TRAIN_N):
        class_l[trial_i] = HRLforNaWater(test=cond[0], reset=RESET)
        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i].softmax(timestep_j)
            class_l[trial_i].get_K(timestep_j)
            class_l[trial_i].get_D(timestep_j)
            class_l[trial_i].get_R(timestep_j)
            class_l[trial_i].get_H(timestep_j)
            class_l[trial_i].get_Q(timestep_j)
            
            ser_num += 1
            
        # Inherit the last Q and H to the next episodes in TRAINs
        inheritQ_inTRAIN[0].append(Q[0, ser_num - 1]), inheritQ_inTRAIN[1].append(Q[1, ser_num - 1])
        inheritH_inTRAIN[0].append(H[0, ser_num - 1]), inheritH_inTRAIN[1].append(H[1, ser_num - 1])

        ser_num += 1

        a_H0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_H1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = H[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[0][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_Q1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = Q[1][(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)]
        a_P0[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].Intake, np.nan)
        a_R[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i : (TIMESTEPS_N + 1) * (trial_i + 1)] = np.append(class_l[trial_i].D, np.nan)

        last_idx = trial_i


    # Inherit Q and H from tha last train to the first test.
    inheritQ_TRAINtoTEST[0].append(Q[0][ser_num - 1]), inheritQ_TRAINtoTEST[1].append(Q[1][ser_num - 1])
    inheritH_TRAINtoTest[0].append(H[0][ser_num - 1]), inheritH_TRAINtoTest[1].append(H[1][ser_num - 1])


    # TEST #################################
    for trial_i in range(TRIALS_TEST_N):
        class_l[trial_i + TRIALS_TRAIN_N] = HRLforNaWater(test=cond[1], reset=RESET)

        for timestep_j in range(TIMESTEPS_N):
            class_l[trial_i + TRIALS_TRAIN_N].softmax(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_K(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_D(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_R(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_H(timestep_j)
            class_l[trial_i + TRIALS_TRAIN_N].get_Q(timestep_j)
            
            ser_num += 1
        
        inheritQ_inTEST[0].append(Q[0, ser_num - 1]), inheritQ_inTEST[1].append(Q[1, ser_num - 1])
        inheritH_inTEST[0].append(H[0, ser_num - 1]), inheritH_inTEST[1].append(H[1, ser_num - 1])
        
        ser_num += 1
        
        a_H0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_H1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = H[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[0][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_Q1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = Q[1][(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N]
        a_P0[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[0], np.nan)
        a_P1[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].P[1], np.nan)
        
        a_a[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].a, np.nan)
        a_Intake[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].Intake, np.nan)        
        a_R[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].R, np.nan)
        a_TDerror[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].TDerror, np.nan)
        a_D[(TIMESTEPS_N + 1) * trial_i + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N : (TIMESTEPS_N + 1) * (trial_i + 1) + (TIMESTEPS_N + 1) * TRIALS_TRAIN_N] = np.append(class_l[trial_i + TRIALS_TRAIN_N].D, np.nan)


    a_H0_d[str(cond)].append(np.array(a_H0)), a_H1_d[str(cond)].append(np.array(a_H1))
    a_Q0_d[str(cond)].append(np.array(a_Q0)), a_Q1_d[str(cond)].append(np.array(a_Q1))
    a_P0_d[str(cond)].append(np.array(a_P0)), a_P1_d[str(cond)].append(np.array(a_P1))    

    a_a_d[str(cond)].append(np.array(a_a)), a_Intake_d[str(cond)].append(np.array(a_Intake))
    a_R_d[str(cond)].append(np.array(a_R)), a_TDerror_d[str(cond)].append(np.array(a_TDerror)), a_D_d[str(cond)].append(np.array(a_D))

# a_a_d[condition][animal][Nan added timestep]

visualize(cond, a_H0, a_H1, a_Q0, a_Q1, a_P0, a_P1, a_Intake, a_a, a_R, a_TDerror, a_D)

print(cond, 'sodium-dep, 750mM salt water test', str(cond))

[1, 3.3] sodium-dep, 750mM salt water test [1, 3.3]


In [28]:
all_data_l = [a_H0_d, a_H1_d, a_Q0_d, a_Q1_d, a_P0_d, a_P1_d, a_Intake_d, a_a_d, a_R_d, a_TDerror_d, a_D_d]

In [29]:
mpl.use("PS")

def BarVisu(a_d, merge):
    bar_width = 0.4
    colors = ['#00BFFF', '#009000', '#FF00FF'] * 2

    if merge:  # Merge 2 tests
        pos_means_d, neg_means_d = {}, {}
        for key_i in a_d.keys():
            pos_means_d[key_i], neg_means_d[key_i] = [], []
            for animal_j in range(ANIMALS_N):
                to_eval_test1 = a_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N)]
                to_eval_test2 = a_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 1)]

                if to_eval_test1 >= 0:
                    pos_means_d[key_i].append(to_eval_test1)
                    neg_means_d[key_i].append(0)
                else:
                    pos_means_d[key_i].append(0)
                    neg_means_d[key_i].append(to_eval_test1)

                if to_eval_test2 >= 0:
                    pos_means_d[key_i][-1] += to_eval_test2
                else:
                    neg_means_d[key_i][-1] += to_eval_test2

        x_l = list(range(len(a_d.keys())))
        means_pos_l, sds_pos_l = [], []
        means_neg_l, sds_neg_l = [], []
        
        for key_i in a_d.keys():
            means_pos_l.append(st.mean(pos_means_d[key_i]))
            sds_pos_l.append(st.stdev(pos_means_d[key_i]))
            means_neg_l.append(st.mean(neg_means_d[key_i]))
            sds_neg_l.append(st.stdev(neg_means_d[key_i]))

        plt.bar(x_l, means_pos_l, yerr=sds_pos_l, capsize=10, color=colors, width=bar_width)
        plt.bar(x_l, means_neg_l, yerr=sds_neg_l, capsize=10, edgecolor=colors, linewidth=2, facecolor='none', color=colors, width=bar_width)
        plt.xticks(x_l, fontsize=8)
        plt.axhline(y=0, color='k', linewidth=1)


    elif not merge:  # Distinguish 2 tests
        pos_means_1_d, neg_means_1_d, pos_means_2_d, neg_means_2_d = {}, {}, {}, {}
        for key_i in a_d.keys():
            pos_means_1_d[key_i], neg_means_1_d[key_i], pos_means_2_d[key_i], neg_means_2_d[key_i] = [], [], [], []
            for animal_j in range(ANIMALS_N):

                to_eval_test1 = a_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N)]
                to_eval_test2 = a_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 1)]

                if to_eval_test1 >= 0:
                    pos_means_1_d[key_i].append(to_eval_test1)
                    neg_means_1_d[key_i].append(0)
                else:
                    pos_means_1_d[key_i].append(0)
                    neg_means_1_d[key_i].append(to_eval_test1)

                if to_eval_test2 >= 0:
                    pos_means_2_d[key_i].append(to_eval_test2)
                    neg_means_2_d[key_i].append(0)
                else:
                    pos_means_2_d[key_i].append(0)
                    neg_means_2_d[key_i].append(to_eval_test2)

        means_pos_1_l, sds_pos_1_l, means_pos_2_l, sds_pos_2_l = [], [], [], []
        means_neg_1_l, sds_neg_1_l, means_neg_2_l, sds_neg_2_l = [], [], [], []

        for key_i in a_d.keys():
            means_pos_1_l.append(st.mean(pos_means_1_d[key_i])), sds_pos_1_l.append(st.stdev(pos_means_1_d[key_i]))
            means_neg_1_l.append(st.mean(neg_means_1_d[key_i])), sds_neg_1_l.append(st.stdev(neg_means_1_d[key_i]))
            means_pos_2_l.append(st.mean(pos_means_2_d[key_i])), sds_pos_2_l.append(st.stdev(pos_means_2_d[key_i]))
            means_neg_2_l.append(st.mean(neg_means_2_d[key_i])), sds_neg_2_l.append(st.stdev(neg_means_2_d[key_i]))

        x_smaller_l = [a - b - 0.01 for a, b in zip(list(range(len(a_d.keys()))), [bar_width / 2] * len(a_d.keys()))]
        x_larger_l = [a + b + 0.01 for a, b in zip(list(range(len(a_d.keys()))), [bar_width / 2] * len(a_d.keys()))]
        x_l = list(range(len(a_d.keys())))

        plt.bar(x_smaller_l, means_pos_1_l, yerr=sds_pos_1_l, capsize=10, color=colors, width=bar_width)
        plt.bar(x_larger_l, means_pos_2_l, yerr=sds_pos_2_l, capsize=10, color=colors, width=bar_width)
        plt.bar(x_smaller_l, means_neg_1_l, yerr=sds_neg_1_l, capsize=10, edgecolor=colors, linewidth=2, facecolor='none', color=colors, width=bar_width)
        plt.bar(x_larger_l, means_neg_2_l, yerr=sds_neg_2_l, capsize=10, edgecolor=colors, linewidth=2, facecolor='none', color=colors, width=bar_width)
        
        plt.xticks(x_l, fontsize=8)
        plt.axhline(y=0, color='k', linewidth=1)


In [30]:
plt.figure()
BarVisu(a_TDerror_d, True)
plt.savefig("TDerror.eps", dpi=600)

In [31]:
plt.figure()
BarVisu(a_R_d, False)

In [32]:
def LickingVisu(lick_d, merge):
    bar_width = 0.4
    colors = ['#00BFFF', '#009000', '#FF00FF'] * 2
    
    if merge:  # Merge 2 tests
        nums_d = {}
        for key_i in lick_d.keys():
            nums_d[key_i] = []
            for animal_j in range(ANIMALS_N):
                nums_d[key_i].append(np.sum(np.nan_to_num(lick_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N) : (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 2)])))

        x_l = list(range(len(lick_d.keys())))

        means_l, sds_l = [], []
        
        for key_i in lick_d.keys():
            means_l.append(st.mean(nums_d[key_i]))
            sds_l.append(st.stdev(nums_d[key_i]))
            
        plt.bar(x_l, means_l, yerr=sds_l, capsize=10, color=colors, width=bar_width)
        #plt.bar(x_l, means_l, yerr=sds_l, capsize=10, edgecolor=colors, linewidth=2, facecolor='none', color=colors, width=bar_width)
        plt.xticks(x_l, fontsize=8)
        plt.axhline(y=0, color='k', linewidth=1)


    else:  # Distinguish 2 tests
        nums_1_d, nums_2_d = {}, {}
        for key_i in lick_d.keys():
            nums_1_d[key_i], nums_2_d[key_i] = [], []
            for animal_j in range(ANIMALS_N):
                nums_1_d[key_i].append(np.sum(np.nan_to_num(lick_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N) : (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 1)])))
                nums_2_d[key_i].append(np.sum(np.nan_to_num(lick_d[key_i][animal_j][(TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 1) : (TIMESTEPS_N + 1) * (TRIALS_TRAIN_N + 2)])))

        means_1_l, sds_1_l = [], []
        means_2_l, sds_2_l = [], []

        for key_i in lick_d.keys():
            means_1_l.append(st.mean(nums_1_d[key_i])), sds_1_l.append(st.stdev(nums_1_d[key_i]))
            means_2_l.append(st.mean(nums_2_d[key_i])), sds_2_l.append(st.stdev(nums_2_d[key_i]))

        x_smaller_l = [a - b - 0.01 for a, b in zip(list(range(len(lick_d.keys()))), [bar_width / 2] * len(lick_d.keys()))]
        x_larger_l = [a + b + 0.01 for a, b in zip(list(range(len(lick_d.keys()))), [bar_width / 2] * len(lick_d.keys()))]
        x_l = list(range(len(lick_d.keys())))

        plt.bar(x_smaller_l, means_1_l, yerr=sds_1_l, capsize=10, color=colors, width=bar_width)
        plt.bar(x_larger_l, means_2_l, yerr=sds_2_l, capsize=10, color=colors, width=bar_width)
        
        plt.xticks(x_l, fontsize=8)
        plt.axhline(y=0, color='k', linewidth=1)
        

In [33]:
plt.figure()
LickingVisu(a_a_d, True)
plt.savefig("lick.eps", dpi=600)